<a href="https://colab.research.google.com/github/Muhammadsulton1/ML/blob/main/nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving nlp.csv to nlp.csv


In [ ]:
# Подавление предупреждений
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('nlp.csv')

In [ ]:
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
print(len(data['text']))

7613


In [ ]:
np.shape(data)

(7613, 5)

In [ ]:
data.describe(include= 'object')

,keyword,location,text
count,7552,5080,7613
unique,221,3341,7503
top,fatalities,USA,11-Year-Old Boy Charged With Manslaughter of T...
freq,45,104,10


In [ ]:
data.columns

Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')

In [ ]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB
None


In [ ]:
#функция для просмотра какой процент пропущенных данных мы имеем
def missing_values_table(data):
        # Общее количество пропущенных значений
        mis_val = data.isnull().sum()
        # Доля пропущенных значений
        mis_val_percent = 100 * data.isnull().sum() / len(data)
        # Таблица с результатом расчета
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        # Переименовываем столбцы
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        # Сортируем по столбцу с долей пропущенных значений в порядке убывания
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        # Вывод сводной информации
        print("Your selected dataframe has " + str(data.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

# Статистика пропущенных значений
missing_values = missing_values_table(data)
missing_values.head(15)

Your selected dataframe has 5 columns.
There are 2 columns that have missing values.


,Missing Values,% of Total Values
location,2533,33.3
keyword,61,0.8


In [ ]:
data = data.drop(['location','keyword','id'],axis = 1)

In [ ]:
data.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
import re

In [ ]:
#применим функцию к нижниму регистру
def capitalizer(string: str) -> str:
    return string.lower()
# Применить функцию
data['text'] = [capitalizer(string) for string in data['text']]

In [ ]:
def remove_emails(x):
     return re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+)',"", x)

In [ ]:
def remove_urls(x):
    return re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , x)

In [ ]:
def remove_rt(x):
    return re.sub(r'\brt\b', '', x).strip()


In [ ]:
def remove_special_chars(x):
    x = re.sub(r'[^\w ]+', "", x)
    x = ' '.join(x.split())
    return x

In [ ]:
def remove_accented_chars(x):
    x = unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return x


In [ ]:
import unicodedata
data['text_clean'] = data['text'].apply(lambda x: remove_emails(x))
data['text_clean'] = data['text_clean'].apply(lambda x: remove_urls(x))
data['text_clean'] = data['text_clean'].apply(lambda x: remove_rt(x))
data['text_clean'] = data['text_clean'].apply(lambda x: remove_special_chars(x))
data['text_clean'] = data['text_clean'].apply(lambda x: remove_accented_chars(x))

In [ ]:
data.head(10)

,text,target,text_clean
0,our deeds are the reason of this #earthquake m...,1,our deeds are the reason of this earthquake ma...
1,forest fire near la ronge sask. canada,1,forest fire near la ronge sask canada
2,all residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...
3,"13,000 people receive #wildfires evacuation or...",1,13000 people receive wildfires evacuation orde...
4,just got sent this photo from ruby #alaska as ...,1,just got sent this photo from ruby alaska as s...
5,#rockyfire update => california hwy. 20 closed...,1,rockyfire update california hwy 20 closed in b...
6,#flood #disaster heavy rain causes flash flood...,1,flood disaster heavy rain causes flash floodin...
7,i'm on top of the hill and i can see a fire in...,1,im on top of the hill and i can see a fire in ...
8,there's an emergency evacuation happening now ...,1,theres an emergency evacuation happening now i...
9,i'm afraid that the tornado is coming to our a...,1,im afraid that the tornado is coming to our area


In [ ]:
# добавления новой колонки к новому фрейму данных , функция apply передает значения всем объектам 
data = data.assign(text_length = data['text_clean'].apply(len))

In [ ]:
data.head()

,text,target,text_clean,text_length
0,our deeds are the reason of this #earthquake m...,1,our deeds are the reason of this earthquake ma...,68
1,forest fire near la ronge sask. canada,1,forest fire near la ronge sask canada,37
2,all residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...,130
3,"13,000 people receive #wildfires evacuation or...",1,13000 people receive wildfires evacuation orde...,62
4,just got sent this photo from ruby #alaska as ...,1,just got sent this photo from ruby alaska as s...,85


In [ ]:
#УДАЛИТЬ ПРОБЕЛЫ
data['text_clean'] = [string.strip() for string in data['text_clean']]

In [ ]:
#удаление точек
data['text_clean'] = [string.replace(".", "") for string in data['text_clean']]

In [ ]:
data.head()

,text,target,text_clean,text_length
0,our deeds are the reason of this #earthquake m...,1,our deeds are the reason of this earthquake ma...,68
1,forest fire near la ronge sask. canada,1,forest fire near la ronge sask canada,37
2,all residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...,130
3,"13,000 people receive #wildfires evacuation or...",1,13000 people receive wildfires evacuation orde...,62
4,just got sent this photo from ruby #alaska as ...,1,just got sent this photo from ruby alaska as s...,85


In [ ]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#токенизация текста
data = data.assign(text_tokenize = data['text_clean'].apply(word_tokenize))

In [ ]:
data.head()

,text,target,text_clean,text_length,text_tokenize
0,our deeds are the reason of this #earthquake m...,1,our deeds are the reason of this earthquake ma...,68,"[our, deeds, are, the, reason, of, this, earth..."
1,forest fire near la ronge sask. canada,1,forest fire near la ronge sask canada,37,"[forest, fire, near, la, ronge, sask, canada]"
2,all residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...,130,"[all, residents, asked, to, shelter, in, place..."
3,"13,000 people receive #wildfires evacuation or...",1,13000 people receive wildfires evacuation orde...,62,"[13000, people, receive, wildfires, evacuation..."
4,just got sent this photo from ruby #alaska as ...,1,just got sent this photo from ruby alaska as s...,85,"[just, got, sent, this, photo, from, ruby, ala..."


In [ ]:
#удаление стоп слов
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words = stopwords.words('english')

In [ ]:
data['text_tokenize'] = data['text_tokenize'].apply(lambda x: [item for item in x if item not in stop_words])

In [ ]:
data.head()

,text,target,text_clean,text_length,text_tokenize
0,our deeds are the reason of this #earthquake m...,1,our deeds are the reason of this earthquake ma...,68,"[deeds, reason, earthquake, may, allah, forgiv..."
1,forest fire near la ronge sask. canada,1,forest fire near la ronge sask canada,37,"[forest, fire, near, la, ronge, sask, canada]"
2,all residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...,130,"[residents, asked, shelter, place, notified, o..."
3,"13,000 people receive #wildfires evacuation or...",1,13000 people receive wildfires evacuation orde...,62,"[13000, people, receive, wildfires, evacuation..."
4,just got sent this photo from ruby #alaska as ...,1,just got sent this photo from ruby alaska as s...,85,"[got, sent, photo, ruby, alaska, smoke, wildfi..."


In [ ]:
#стемминг 
import nltk
from nltk.stem import SnowballStemmer

In [ ]:
snowball = SnowballStemmer("english")

In [ ]:
data['text_tokenize'] = data['text_tokenize'].apply(lambda x: [snowball.stem(y) for y in x])

In [ ]:
data.head()

,text,target,text_clean,text_length,text_tokenize
0,our deeds are the reason of this #earthquake m...,1,our deeds are the reason of this earthquake ma...,68,"[deed, reason, earthquak, may, allah, forgiv, us]"
1,forest fire near la ronge sask. canada,1,forest fire near la ronge sask canada,37,"[forest, fire, near, la, rong, sask, canada]"
2,all residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...,130,"[resid, ask, shelter, place, notifi, offic, ev..."
3,"13,000 people receive #wildfires evacuation or...",1,13000 people receive wildfires evacuation orde...,62,"[13000, peopl, receiv, wildfir, evacu, order, ..."
4,just got sent this photo from ruby #alaska as ...,1,just got sent this photo from ruby alaska as s...,85,"[got, sent, photo, rubi, alaska, smoke, wildfi..."


In [ ]:
#лемитизация слов
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
data['text_tokenize'] = data['text_tokenize'].apply(lambda x: [lemmatizer.lemmatize(y, pos='v') for y in x])

In [ ]:
data.head()

,text,target,text_clean,text_length,text_tokenize
0,our deeds are the reason of this #earthquake m...,1,our deeds are the reason of this earthquake ma...,68,"[deed, reason, earthquak, may, allah, forgiv, us]"
1,forest fire near la ronge sask. canada,1,forest fire near la ronge sask canada,37,"[forest, fire, near, la, rong, sask, canada]"
2,all residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...,130,"[resid, ask, shelter, place, notifi, offic, ev..."
3,"13,000 people receive #wildfires evacuation or...",1,13000 people receive wildfires evacuation orde...,62,"[13000, peopl, receiv, wildfir, evacu, order, ..."
4,just got sent this photo from ruby #alaska as ...,1,just got sent this photo from ruby alaska as s...,85,"[get, send, photo, rubi, alaska, smoke, wildfi..."


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
data = data.assign(text_str = data['text_tokenize'].apply(str))

In [ ]:
data.head()

,text,target,text_clean,text_length,text_tokenize,text_str
0,our deeds are the reason of this #earthquake m...,1,our deeds are the reason of this earthquake ma...,68,"[deed, reason, earthquak, may, allah, forgiv, us]","['deed', 'reason', 'earthquak', 'may', 'allah'..."
1,forest fire near la ronge sask. canada,1,forest fire near la ronge sask canada,37,"[forest, fire, near, la, rong, sask, canada]","['forest', 'fire', 'near', 'la', 'rong', 'sask..."
2,all residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...,130,"[resid, ask, shelter, place, notifi, offic, ev...","['resid', 'ask', 'shelter', 'place', 'notifi',..."
3,"13,000 people receive #wildfires evacuation or...",1,13000 people receive wildfires evacuation orde...,62,"[13000, peopl, receiv, wildfir, evacu, order, ...","['13000', 'peopl', 'receiv', 'wildfir', 'evacu..."
4,just got sent this photo from ruby #alaska as ...,1,just got sent this photo from ruby alaska as s...,85,"[get, send, photo, rubi, alaska, smoke, wildfi...","['get', 'send', 'photo', 'rubi', 'alaska', 'sm..."


In [ ]:
#выбираем 'text_str' потому что он очищеный приведен к начальной форме удалены стоп слова проведена лексимизация текста итп 
X = data['text_str']
y = data['target']

In [ ]:
vectorizer = CountVectorizer()
# подгоняем модель под векторизацию
vectorizer.fit(X)
# преобразуем нашу разряженную матрицу 
X = vectorizer.transform(X)
print('размер разряженной матрицы: ', X.shape)
print('количество ненулевых вхождений: ', X.nnz)

размер разряженной матрицы:  (7613, 15016)
количество ненулевых вхождений:  68478


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.dummy import DummyClassifier

In [ ]:
#разбиение на тестовый и тренировочный набор данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [ ]:
dummy = DummyClassifier(strategy='uniform', random_state=100)

In [ ]:
dummy.fit(X_train, y_train)
dummy.score(X_test, y_test)

0.5042678923177938

In [ ]:
# создание модели случайного леса
rfc = RandomForestClassifier(criterion = 'entropy',oob_score=True ,n_jobs=-1, random_state=105)
# начинаем подгонку модели на тренировачных данных
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=True, random_state=105, verbose=0,
                       warm_start=False)

In [ ]:
# предсказать значения тестовой выборки
predictions_rfc = rfc.predict(X_test)
# посмотреть на точность предсказания 
f1_score(y_test,predictions_rfc)

0.7118942731277533

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
adaboost = AdaBoostClassifier(random_state= 120)

In [ ]:
model_adaboost = adaboost.fit(X_train, y_train)

In [ ]:
# предсказать значения тестовой выборки
predictions_adaboost = model_adaboost.predict(X_test)
# посмотреть на точность предсказания 
f1_score(y_test, predictions_adaboost,average='micro')

0.7649376231122784

In [ ]:
logistic = LogisticRegression(random_state=241, n_jobs=-1)

In [ ]:
logistic.fit(X_train, y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=241, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
# предсказать значения тестовой выборки
predictions_logistic = logistic.predict(X_test)
# посмотреть на точность предсказания 
f1_score(y_test, predictions_logistic,average='micro')

0.7826657912015759

In [ ]:
# n =3  уже подобранное число соседа
knn = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)

In [ ]:
knn.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=3, p=2,
                     weights='uniform')

In [ ]:
# предсказать значения тестовой выборки
predictions_knn = knn.predict(X_test)
# посмотреть на точность предсказания 
f1_score(y_test, predictions_knn,average='micro')

0.7235718975705844

In [ ]:
from sklearn.svm import SVC

In [ ]:
#подобранные параметры
svc = SVC(kernel= 'rbf',C=1,gamma=0.1)

In [ ]:
model_svc = svc.fit(X_train, y_train)

In [ ]:
# предсказать значения тестовой выборки
predictions_svc = model_svc.predict(X_test)
# посмотреть на точность предсказания 
f1_score(y_test, predictions_svc,average='micro')

0.7925147734734077

In [ ]:
decisiontree = DecisionTreeClassifier(criterion = 'entropy', random_state = 244)

In [ ]:
model_tree = decisiontree.fit(X_train, y_train)

In [ ]:
# предсказать значения тестовой выборки
predictions_tree = model_tree.predict(X_test)
# посмотреть на точность предсказания 
f1_score(y_test, predictions_tree,average='micro')

0.7360472751149048

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
classifier = MultinomialNB()
X_class = X_train.toarray()

In [ ]:
model_bayes = classifier.fit(X_class, y_train)

In [ ]:
# предсказать значения тестовой выборки
predictions_bayes = model_bayes.predict(X_test)
# посмотреть на точность предсказания 
f1_score(y_test, predictions_bayes,average='micro')

0.7859487852921865

In [ ]:
from sklearn.naive_bayes import BernoulliNB

In [ ]:
classifier_bernulli = BernoulliNB()

In [ ]:
model_bernulli = classifier_bernulli.fit(X_class,y_train)

In [ ]:
# предсказать значения тестовой выборки
predictions_bernulli = model_bernulli.predict(X_test)
# посмотреть на точность предсказания 
f1_score(y_test, predictions_bernulli,average='micro')

0.7977675640183848

ОПТИМИЗАЦИЯ НАШИХ АЛГОРИТМОВ

In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline

In [ ]:
penalty = ['l1','l2']
C = np.logspace(0,4,10)
hyperparametrs = dict(C=C, penalty=penalty)

In [ ]:
grid_search_logit = GridSearchCV(logistic, hyperparametrs, cv = 5, verbose = 0)

In [ ]:
best_model_logit = grid_search_logit.fit(X_train, y_train)

In [ ]:
print('Лучший штраф:', best_model_logit.best_estimator_.get_params()['penalty'])
print('Лучший C:', best_model_logit.best_estimator_.get_params()['C'])

Лучший штраф: l2
Лучший C: 1.0


In [ ]:
logistic_optim = LogisticRegression(penalty='l2', C=1,n_jobs=-1,random_state=220)

In [ ]:
model_logit_optim = logistic_optim.fit(X_train,y_train)

In [ ]:
# предсказать значения тестовой выборки
predictions_logit_optim = model_logit_optim.predict(X_test)
# посмотреть на точность предсказания 
f1_score(y_test, predictions_logit_optim,average='micro')

0.7826657912015759

ОПТИМИЗАЦИЯ К - БЛИЖАЙШИХ СОСЕДЕЙ


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion

In [ ]:
n_neighbors = [1,2,3,4,5,6,7,8,9,10]
param_grid = {'n_neighbors':n_neighbors,}

In [ ]:
gs = GridSearchCV(knn, param_grid, cv = 5, verbose=1, n_jobs=-1)

In [ ]:
best_model_knn = gs.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   10.5s finished


In [ ]:
best_model_knn.best_params_

{'n_neighbors': 3}

In [ ]:
knn_optim = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)

In [ ]:
model_knn_optim = knn_optim.fit(X_train,y_train)

In [ ]:
# предсказать значения тестовой выборки
predictions_knn_optim = model_knn_optim.predict(X_test)
# посмотреть на точность предсказания 
f1_score(y_test, predictions_knn_optim,average='micro')

0.7235718975705844

ОПТИМИЗАЦИЯ МЕТОДА ОПОРНО МАШИННЫХ ВЕКТОРОВ

In [ ]:
param_grid_svm = {'C': [1,2,3,4,5,10,100],
                  'gamma': [1,0.1, 0.01, 0.001, 0.0001],
                  'kernel':['rbf','linear','poly','sigmoid']}

In [ ]:
grid_svm = GridSearchCV(svc, param_grid_svm,cv=5, verbose=3, n_jobs=-1)

In [ ]:
best_model_svm = grid_svm.fit(X_train,y_train)

In [ ]:
best_model_svm.best_params_

In [ ]:
svc_optim = SVC(C=1, kernel='rbf',gamma=0.1, random_state=202)

In [ ]:
model_svc_optim = svc_optim.fit(X_train,y_train)

In [ ]:
# предсказать значения тестовой выборки
predictions_svc_optim = model_svc_optim.predict(X_test)
# посмотреть на точность предсказания 
f1_score(y_test, predictions_svc_optim,average='micro')

0.7925147734734077

ОПТИМИЗАЦИЯ СЛУЧАЙНОГО ЛЕСА


In [ ]:
#решетчетый поиск для оптимизации случайных лесов
n_estimators = [300,500,700,800]
param_grid = {'n_estimators':n_estimators,
              'min_samples_leaf':range(2,9),
              'min_samples_split':range(2,9),
              'max_depth': range(1,71)}#глубина дерева от 1 до 51 с шагом 1

In [ ]:
grid_search_rfc = GridSearchCV(rfc, param_grid, cv = 3, verbose=1, n_jobs=-1)

In [ ]:
best_model_rfc = grid_search_rfc.fit(X_train,y_train)

Fitting 3 folds for each of 13720 candidates, totalling 41160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed: 20.5min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed: 33.3min
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed: 50.2min
[Parallel(n_jobs=-1)]: Done 2446 tasks      | elapsed: 72.3min
[Parallel(n_jobs=-1)]: Done 3196 tasks      | elapsed: 100.4min
[Parallel(n_jobs=-1)]: Done 4046 tasks      | elapsed: 135.9min
[Parallel(n_jobs=-1)]: Done 4996 tasks      | elapsed: 180.2min
[Parallel(n_jobs=-1)]: Done 6046 tasks      | elapsed: 234.2min


In [ ]:
best_model_rfc.best_params_

NameError: ignored

In [ ]:
model_rfc = RandomForestClassifier(criterion= 'entropy', n_estimators=500, max_depth = 50,min_samples_leaf=3,random_state=300, n_jobs=-1 )

In [ ]:
model_rfc_optim = model_rfc.fit(X_train,y_train)

In [ ]:
# предсказать значения тестовой выборки
predictions_rfc_optim = model_rfc_optim.predict(X_test)
# посмотреть на точность предсказания 
f1_score(y_test, predictions_rfc_optim,average='micro')

0.757715036112935

ОПТИМИЗАЦИЯ ДЕРЕВА ПРИНЯТИЯ РЕШЕНИЙ


In [ ]:
param_grid_clf = {
              'max_depth': range(1,71),#глубина дерева от 1 до 71 
              'min_samples_split': range(2,11),
              'min_samples_leaf':range(2,8)}

In [ ]:
grid_search_tree = GridSearchCV(decisiontree, param_grid_clf, cv = 4, verbose=1, n_jobs=-1)

In [ ]:
best_model_tree = grid_search_tree.fit(X_train,y_train)

Fitting 3 folds for each of 3780 candidates, totalling 11340 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  70 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 370 tasks      | elapsed:   16.5s
[Parallel(n_jobs=-1)]: Done 870 tasks      | elapsed:   41.4s
[Parallel(n_jobs=-1)]: Done 1570 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 2470 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 3570 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 4870 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done 6370 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done 8070 tasks      | elapsed: 14.3min
[Parallel(n_jobs=-1)]: Done 9970 tasks      | elapsed: 20.1min
[Parallel(n_jobs=-1)]: Done 11340 out of 11340 | elapsed: 24.6min finished


In [ ]:
best_model_tree.best_params_

{'max_depth': 66, 'min_samples_leaf': 6, 'min_samples_split': 2}

In [ ]:
model_decisiontree = DecisionTreeClassifier(criterion='entropy', max_depth=66,min_samples_leaf=6,min_samples_split=2)

In [ ]:
model_decisiontree_optim = model_decisiontree.fit(X_train, y_train)

In [ ]:
# предсказать значения тестовой выборки
predictions_decision_optim = model_decisiontree_optim.predict(X_test)
# посмотреть на точность предсказания 
f1_score(y_test, predictions_decision_optim,average='micro')

0.7747866053841101

Кросс валидация логистической регрессии 

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold,cross_val_score


In [ ]:
kf_logit = KFold(n_splits = 10,shuffle = True,random_state = 41)

In [ ]:
cv_results_logit = cross_val_score(logistic,X,y,cv = kf_logit,scoring = "f1",n_jobs = -1)

In [ ]:
cv_results_logit.mean()

0.748400070177481

КРОСС ВАЛИДАЦИЯ ДЕРЕВА ПРИНЯТИЯ РЕШЕНИЙ

In [ ]:
kf_tree = KFold(n_splits = 10,shuffle = True,random_state = 42)

In [ ]:
cv_results_tree = cross_val_score(model_decisiontree,X,y,cv = kf_tree,scoring = "f1",n_jobs = -1)

In [ ]:
cv_results_tree.mean()

0.7003312104283317

КРОСС ВАЛИДАЦИЯ СЛУЧАЙНЫХ ЛЕСОВ

In [ ]:
kf_rfc = KFold(n_splits = 10,shuffle = True,random_state = 43)

In [ ]:
cv_results_rfc = cross_val_score(model_rfc_optim,X,y,cv = kf_rfc,scoring = "f1",n_jobs = -1)

In [ ]:
cv_results_rfc.mean()

0.66168118210824

КРОСС ВАЛИДАЦИЯ Метода машинна опорных векторов


In [ ]:
kf_svc = KFold(n_splits=10,shuffle=True,random_state=44)
cv_results_svc = cross_val_score(svc,X,y,cv=kf_svc, scoring="f1",n_jobs=-1)

In [ ]:
cv_results_svc.mean()

0.7425950547309099

КРОСС ВАЛИДАЦИЯ АДАБУСТКЛАССИФАЙРА

In [ ]:
kf_adaboost = KFold(n_splits=10,shuffle=True,random_state=45)

In [ ]:
sv_results_adaboost=cross_val_score(adaboost,X,y,cv=kf_adaboost,scoring="f1",n_jobs=-1)

In [ ]:
sv_results_adaboost.mean()

0.6835203584358978

КРОСС ВАЛИДАЦИЯ ПОЛИНОМИАЛЬНОГО БАЙСЕВОГО КЛАССИФИКАТОРА

In [ ]:
kf_multi = KFold(n_splits=10,shuffle=True,random_state=45)

In [ ]:
sv_results_multi = cross_val_score(classifier,X,y,cv=kf_multi,scoring="f1",n_jobs=-1)

In [ ]:
sv_results_multi.mean()

0.7578488423840888

КРОСС ВАЛИДАЦИЯ БЕРНУЛИ БАЙСОВОГО КЛАССИФИКАТОРА

In [ ]:
kf_bernulli = KFold(n_splits=10,shuffle=True, random_state=47)

In [ ]:
sv_results_bernulli = cross_val_score(classifier_bernulli,X,y,cv=kf_bernulli,scoring="f1",n_jobs=-1)

In [ ]:
sv_results_bernulli.mean()

0.7479081986720517

КРОСС ВАЛИДАЦИЯ К-БЛИЖАЙШИХ СОСЕДЕЙ

In [ ]:
kf_knn = KFold(n_splits=10,shuffle=True, random_state=48)

In [ ]:
sv_results_knn = cross_val_score(classifier_bernulli,X,y,cv=kf_knn,scoring="f1",n_jobs=-1)

In [ ]:
sv_results_knn.mean()

0.7481491800869694

In [ ]:
from keras import models
from keras import layers 

In [ ]:
np.random.seed(0)

In [ ]:
network = models.Sequential()

In [ ]:
network.add(layers.Dense(units=16,activation="relu",input_shape=(1000,)))

In [ ]:
network.add(layers.Dense(units=16,activation="relu"))

In [ ]:
network.add(layers.Dense(units=1, activation="sigmoid"))

In [ ]:
network.compile(
loss="binary_crossentropy", # Перекрестная энтропия
optimizer="rmsprop", # Распространение среднеквадратической ошибки
metrics=["f1"]) # Точностный показатель результативности